# Lab 5: Adversarial Training / TangentProp / Tangent Classifier

Simplified and cleaned code from your lab manual.

In [ ]:
# Lab 5: Adversarial training, Tangent Prop & Tangent-distance classifier (Fashion-MNIST subset)
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Layer
import matplotlib.pyplot as plt

# Load subset for speed
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
subset = 1000
x_train = x_train[:subset] / 255.0; y_train = y_train[:subset]
x_test = x_test[:200] / 255.0; y_test = y_test[:200]

def create_model():
    model = Sequential([Flatten(input_shape=(28,28)), Dense(64, activation='relu'), Dense(10, activation='softmax')])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def generate_adversarial_examples(x, epsilon=0.1):
    np.random.seed(100)
    perturbations = np.sign(np.random.randn(*x.shape))
    x_adv = x + epsilon * perturbations
    return np.clip(x_adv, 0, 1)

class TangentProp(Layer):
    def call(self, x):
        perturbation = tf.random.normal(shape=tf.shape(x), stddev=0.1)
        return x + perturbation

def tangent_distance(x1, x2):
    return np.linalg.norm(x1 - x2)

# Adversarial training
model = create_model()
x_adv = generate_adversarial_examples(x_train)
x_combined = np.concatenate([x_train, x_adv])
y_combined = np.concatenate([y_train, y_train])
history_adv = model.fit(x_combined, y_combined, epochs=5, validation_data=(x_test, y_test), verbose=0)

# TangentProp model (adds the layer and retrains)
model_tangent = create_model()
# attach tangent prop layer by wrapping input preprocessing
model_tangent.add(TangentProp())
history_tangent = model_tangent.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test), verbose=0)

loss, acc = model_tangent.evaluate(x_test, y_test, verbose=0)
print(f'TangentProp model accuracy: {acc*100:.2f}%')

# Tangent-distance classifier (nearest neighbor style)
def classify_with_tangent_distance(x_train, y_train, x_test):
    x_train_flat = x_train.reshape(x_train.shape[0], -1)
    x_test_flat = x_test.reshape(x_test.shape[0], -1)
    y_pred = []
    for x in x_test_flat:
        distances = [tangent_distance(x, x_train_flat[i]) for i in range(len(x_train_flat))]
        idx = np.argmin(distances)
        y_pred.append(y_train[idx])
    return np.array(y_pred)

y_pred = classify_with_tangent_distance(x_train, y_train, x_test)
acc_td = np.mean(y_pred == y_test)
print(f'Tangent-distance classifier accuracy: {acc_td*100:.2f}%')

# Plot training losses
plt.plot(history_adv.history['loss'], label='Adversarial Loss')
plt.plot(history_tangent.history['loss'], label='TangentProp Loss')
plt.xlabel('Epochs'); plt.ylabel('Loss'); plt.legend(); plt.title('Training Loss'); plt.show()

In [ ]:
# End of Lab 5